# PROYECTO FINAL: EJERCICIO C2


Usando las funciones desarrolladas en el ejercicio anterior, anota la base de datos facilitada. Después, separa dicha base de datos en entrenamiento, test y validación, utilizando un porcentaje de imágenes para cada conjunto que sea coherente. Re-entrena una arquitectura de detección de implantes y otra de identificación de implantes (puedes usar cualquiera de las arquitecturas vistas en clase).

Muestra las métricas de test y validación. Comenta si has usado data augmentation, de qué tipo, y, tal como se indicaba en la información al inicio del documento, explica qué arquitectura e hiperparametros has utilizado.

In [ ]:
# ==============================================================================
# PROYECTO FINAL: BLOQUE C2 (Deep Learning) - EJECUCIÓN COMPLETA E INTEGRADA
# Basado íntegramente en las metodologías de la Sesión P10.
# ==============================================================================

import os
import random
import shutil
import torch
import torchvision
import numpy as np
from PIL import Image
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# ------------------------------------------------------------------------------
# 1. CONFIGURACIÓN INICIAL Y SEMILLA (P10)
# ------------------------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Entorno configurado. Dispositivo: {device}")

# ------------------------------------------------------------------------------
# 2. PARTICIÓN FÍSICA Y SINCRONIZACIÓN DEL DATASET (P10)
# ------------------------------------------------------------------------------
# Orígenes desde MATLAB
orig_imgs = 'dataset_implantes/PNGImages/'
orig_masks = 'dataset_implantes/PedMasks/'
orig_labels = 'yolo_dataset/labels/'
base_data = 'final_split_dataset/'

# Crear subcarpetas para sincronizar train/val/test
splits = ['train', 'val', 'test']
for s in splits:
    for sub in ['images', 'masks', 'labels']:
        os.makedirs(os.path.join(base_data, s, sub), exist_ok=True)

# Listar y repartir (80/10/10)
ficheros = [f for f in os.listdir(orig_imgs) if f.endswith('.png')]
random.shuffle(ficheros)
total = len(ficheros)
i_train, i_val = int(0.8 * total), int(0.9 * total)

for i, f in enumerate(ficheros):
    # Determinar split
    if i < i_train: s = 'train'
    elif i < i_val: s = 'val'
    else: s = 'test'
    
    # Copiar fotos, máscaras y etiquetas .txt de YOLO
    shutil.copy(os.path.join(orig_imgs, f), os.path.join(base_data, s, 'images', f))
    shutil.copy(os.path.join(orig_masks, f), os.path.join(base_data, s, 'masks', f))
    shutil.copy(os.path.join(orig_labels, f.replace('.png', '.txt')), os.path.join(base_data, s, 'labels', f.replace('.png', '.txt')))

print("Partición física completada y sincronizada al 80/10/10.")

# ------------------------------------------------------------------------------
# 3. GENERACIÓN DEL ARCHIVO .YAML PARA YOLO (P10)
# ------------------------------------------------------------------------------
path_abs = os.path.abspath(base_data)
yaml_content = f"""
path: {path_abs}
train: train/images
val: val/images
test: test/images
nc: 1
names: ['implante']
"""
with open("implantes.yaml", "w") as f:
    f.write(yaml_content)

# ------------------------------------------------------------------------------
# 4. DEFINICIÓN DEL DATASET PARA MASK R-CNN (P10)
# ------------------------------------------------------------------------------
class ImplanteDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.root, "images", self.imgs[idx])).convert("RGB")
        mask = np.array(Image.open(os.path.join(self.root, "masks", self.masks[idx])))
        obj_ids = np.unique(mask)[1:]
        masks = torch.as_tensor(mask == obj_ids[:, None, None], dtype=torch.uint8)
        
        boxes = []
        for i in range(len(obj_ids)):
            pos = np.where(masks[i])
            boxes.append([np.min(pos[1]), np.min(pos[0]), np.max(pos[1]), np.max(pos[0])])
        
        target = {
            "boxes": torch.as_tensor(boxes, dtype=torch.float32),
            "labels": torch.ones((len(obj_ids),), dtype=torch.int64),
            "masks": masks,
            "image_id": torch.tensor([idx]),
            "area": torch.as_tensor([(b[2]-b[0])*(b[3]-b[1]) for b in boxes], dtype=torch.float32),
            "iscrowd": torch.zeros((len(obj_ids),), dtype=torch.int64)
        }
        if self.transforms: img = self.transforms(img)
        return img, target

    def __len__(self): return len(self.imgs)

def collate_fn(batch): return tuple(zip(*batch))
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# Crear DataLoaders apuntando a las carpetas físicas sincronizadas
train_loader = DataLoader(ImplanteDataset(os.path.join(base_data, 'train'), transform), batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(ImplanteDataset(os.path.join(base_data, 'val'), transform), batch_size=2, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ImplanteDataset(os.path.join(base_data, 'test'), transform), batch_size=2, shuffle=False, collate_fn=collate_fn)

# ------------------------------------------------------------------------------
# 5. ENTRENAMIENTO DE MODELOS (P10)
# ------------------------------------------------------------------------------
# A) YOLOv11 (Detección)
print("\n--- Iniciando entrenamiento YOLOv11 ---")
yolo_model = YOLO('yolo11n.pt')
yolo_model.train(data='implantes.yaml', epochs=15, imgsz=640, freeze=10, project='PF', name='yolo_det')

# B) Mask R-CNN (Identificación)
print("\n--- Iniciando entrenamiento Mask R-CNN ---")
mask_model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
in_f = mask_model.roi_heads.box_predictor.cls_score.in_features
mask_model.roi_heads.box_predictor = FastRCNNPredictor(in_f, 2)
in_f_m = mask_model.roi_heads.mask_predictor.conv5_mask.in_channels
mask_model.roi_heads.mask_predictor = MaskRCNNPredictor(in_f_m, 256, 2)
mask_model.to(device)

opt = torch.optim.SGD(mask_model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
for epoch in range(10):
    mask_model.train()
    for imgs, targets in train_loader:
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = mask_model(imgs, targets)
        loss = sum(loss for loss in loss_dict.values())
        opt.zero_grad(); loss.backward(); opt.step()
    print(f"Época {epoch+1} completada.")

# ------------------------------------------------------------------------------
# 6. EVALUACIÓN FINAL (P10)
# ------------------------------------------------------------------------------
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    metric = MeanAveragePrecision(iou_type="bbox")
    for imgs, targets in loader:
        imgs = [img.to(device) for img in imgs]
        outs = model(imgs)
        metric.update([{k: v.cpu() for k, v in o.items()} for o in outs], [{k: v.cpu() for k, v in t.items()} for t in targets])
    return metric.compute()

print("\nResultados Finales (mAP@50):")
res = evaluate(mask_model, test_loader)
print(f"Mask R-CNN Test mAP: {res['map_50']:.3f}")